# Chrome Built-in AI Demo in JupyterLab and JupyterLite Python Notebook

This notebook demonstrates the Python wrapper for Chrome's built-in AI Prompt API.

https://developer.chrome.com/docs/ai/prompt-api

**Requirements:**
- Chrome browser with Prompt API enabled
- Running in JupyterLab, Jupyter Notebook, or JupyterLite (statically served, all execution locally in browser)

## For developers: Using the Prompt API
* Open Chrome and go to `chrome://flags`.
* Enable the `#prompt-api-for-gemini-nano` flag.
* Enable the `#optimization-guide-on-device-model` flag. If you see a "BypassPerfRequirement" option, select it.
* Restart Chrome.

In [1]:
%pip install -q anywidget ipywidgets wiki3_ai>0.3.1

Note: you may need to restart the kernel to use updated packages.


In [2]:
%gui asyncio

In [3]:
# Import the library
from wiki3_ai import LanguageModel

In [4]:
LanguageModel.widget()

## Check Availability

First, let's check if the Prompt API is available in your browser.

In [6]:
# Check if the API is available
await LanguageModel.availability()

'available'

## Get Model Parameters

Let's see what parameters the model supports.

In [7]:
params = await LanguageModel.params()

if params:
    print(f"Default temperature: {params.default_temperature}")
    print(f"Max temperature: {params.max_temperature}")
    print(f"Default top-K: {params.default_top_k}")
    print(f"Max top-K: {params.max_top_k}")
else:
    print("Model parameters not available")

params

Default temperature: 1
Max temperature: 2
Default top-K: 3
Max top-K: 128


LanguageModelParams(default_top_k=3, max_top_k=128, default_temperature=1, max_temperature=2)

## Create LanguageModel Session

In [8]:
lm = await LanguageModel.create()
lm

## Simple Prompt

Create a session and send a simple prompt.

In [9]:
# Send a prompt
result = await lm.prompt("Write a haiku about Python programming.")
print(result)

Clean, readable code,
Logic flows, a gentle stream,
Power in each line. 






## System Prompt

Use a system prompt to set the context for the conversation.

In [10]:
# Create a session with a system prompt
assistant_session = await LanguageModel.create(
    {
        "initialPrompts": [
            {
                "role": "system",
                "content": "You are a helpful Python programming assistant who gives concise answers.",
            }
        ]
    }
)

# Ask a question
result = await assistant_session.prompt("How do I read a CSV file in Python?")
print(result)

```python
import csv

with open('your_file.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        print(row)
```

Replace `'your_file.csv'` with the actual filename.  `csv.reader` iterates through rows.



## Streaming Response

Stream the response as it's generated.

In [11]:
print("Response: ", end="")
async for chunk in assistant_session.prompt_streaming("Tell me a short joke about computers."):
    print(chunk, end="", flush=True)
print()

Response: 


## Multi-turn Conversation

Have a conversation with context maintained across prompts.

In [12]:
# First message
result1 = await assistant_session.prompt("What is a list comprehension?")
print("Assistant:", result1)
print()

# Follow-up message (the assistant remembers the context)
result2 = await assistant_session.prompt("Can you show me an example?")
print("Assistant:", result2)

Assistant: A list comprehension is a concise way to create lists in Python. It's a more compact alternative to loops when generating lists based on existing iterables.

Example:

```python
squares = [x**2 for x in range(10)]  # Creates a list of squares from 0 to 9
```


Assistant: ```python
numbers = [1, 2, 3, 4, 5]

# Create a new list containing the squares of each number
squares = [x**2 for x in numbers]

print(squares)  # Output: [1, 4, 9, 16, 25]

#Create a list of even numbers
even_numbers = [x for x in numbers if x % 2 == 0]

print(even_numbers) # Output: [2, 4]
```

The general format is: `[expression for item in iterable if condition]`



## Check Token Usage

Monitor how many tokens you've used.

In [13]:
print(f"Current usage: {assistant_session.input_usage}/{assistant_session.input_quota} tokens")

# Measure how many tokens a prompt would use
usage = await assistant_session.measure_input_usage("What is machine learning?")
print(f"\nThis prompt would use approximately {usage} tokens")

Current usage: 355/9216 tokens

This prompt would use approximately 13 tokens


## Structured Output

Use JSON schema to get structured responses.

In [14]:
import json

# Define a JSON schema
schema = {
    "type": "object",
    "required": ["sentiment", "score"],
    "properties": {
        "sentiment": {"type": "string", "enum": ["positive", "negative", "neutral"]},
        "score": {"type": "number", "minimum": 0, "maximum": 1},
    },
}

# Get structured response
result = await lm.prompt(
    "Analyze this review: The product exceeded all my expectations! Absolutely amazing!",
    {"responseConstraint": schema},
)

# Parse the JSON response
data = json.loads(result)
print(f"Sentiment: {data['sentiment']}")
print(f"Score: {data['score']}")

Sentiment: positive
Score: 0.95


## Session Cloning

Clone a session to create different conversation branches.

In [15]:
# Start a story
story_session = await LanguageModel.create(
    {"initialPrompts": [{"role": "system", "content": "You are a creative storyteller."}]}
)

await story_session.prompt("Once upon a time, there was a dragon.")

# Create two different story branches
branch1 = await story_session.clone()
branch2 = await story_session.clone()

result1 = await branch1.prompt("The dragon was friendly and helpful.")
print("Branch 1:", result1)
print()

result2 = await branch2.prompt("The dragon was fierce and terrifying.")
print("Branch 2:", result2)

Branch 1: Once upon a time, there was a dragon named Lumina. But she wasn't like the dragons in old legends. Lumina wasn’t a fearsome beast hoarding gold or breathing fire. She was a remarkably friendly dragon, and incredibly helpful to anyone who crossed her path. 

Lumina lived in the Whispering Caves, a cavern system carved into the heart of the Everwood, a forest known for its ancient trees and even more ancient secrets. Her scales shimmered not with fiery hues, but with the soft, pearlescent colours of a sunrise - lavender, rose, and a touch of gold, reflecting the kindness she carried within. 

She’d actively seek out travelers, not to threaten them, but to offer assistance. If a traveler’s wagon broke down, Lumina, with a gentle nudge of her snout, could easily right it. If a lost child wandered into the woods, Lumina’s warm glow would illuminate the path home.  She'd patiently listen to their troubles, offering a comforting rumble of her voice and a thoughtful suggestion gleane

## Cleanup

Destroy sessions when you're done to free up resources.

In [16]:
await lm.destroy()
await assistant_session.destroy()
await story_session.destroy()
await branch1.destroy()
await branch2.destroy()

print("✅ All sessions destroyed")

✅ All sessions destroyed
